In [1]:
import cv2
import face_recognition
import os
import numpy as np
from collections import defaultdict


known_faces_dir will be C:/Users/ajair/Desktop/github/AiSystemLabs/face_recognition_project/known_faces

In [2]:
def load_known_faces(known_faces_dir):
    """Load and encode known faces from directory structure
    Expected structure:
    known_faces/
        person1_name/
            image1.jpg
            image2.jpg
        person2_name/
            image1.jpg
    """
    known_faces = defaultdict(list)  # Automatically creates empty list for new keys
    
    # Check if directory exists
    if not os.path.exists(known_faces_dir):
        print(f"Directory {known_faces_dir} not found!")
        return {}
    
    # Iterate through each person's directory
    for person_name in os.listdir(known_faces_dir): # os.listdir() lists all files and directories in the given directory in list format
        person_dir = os.path.join(known_faces_dir, person_name) #known_faces/Ajai_Raj 
        if not os.path.isdir(person_dir): # checks if each path is a directory (folder) otherwise skips
            continue # means if the path is not a directory, skip the rest of the code and go to the next person
        
        # Load each image for this person
        for image_file in os.listdir(person_dir):
            if not image_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                continue
            
            image_path = os.path.join(person_dir, image_file)
            try:
                # Load and encode face
                image = face_recognition.load_image_file(image_path)
                face_encodings = face_recognition.face_encodings(image)
                
                if not face_encodings:
                    print(f"No face found in {image_path}")
                    continue
                
                # Add the first found face encoding
                known_faces[person_name].append(face_encodings[0])
                print(f"Loaded face from {image_path}")
                
            except Exception as e:
                print(f"Error loading {image_path}: {str(e)}")
    
    return known_faces


os.path.abspath() gets the absolute (full) path of a directory
The empty string '' means "current directory"
So this line gets the full path of where your notebook is running
For example, if your notebook is in C:/Users/ajair/Desktop/github/AiSystemLabs/face_recognition_project, that's what notebook_dir will contain

In [3]:
# Get the notebook's directory path
notebook_dir = os.path.abspath('') # os.path.abspath() gets the absolute (full) path of a directory 
known_faces_dir = os.path.join(notebook_dir, "known_faces")


It takes the notebook_dir (your current directory) and adds "known_faces" to it
So if notebook_dir is C:/Users/ajair/Desktop/github/AiSystemLabs/face_recognition_project
Then known_faces_dir will be C:/Users/ajair/Desktop/github/AiSystemLabs/face_recognition_project/known_faces

In [4]:
# Initialize video capture (0 is usually the built-in webcam)
video_capture = cv2.VideoCapture(0)

# Load known faces
known_faces = load_known_faces(known_faces_dir)

# Check if we found any faces
if not known_faces:
    print("No known faces found! Please add person folders with training images")
    print("Expected structure:")
    print("known_faces/")
    print("    person1_name/")
    print("        image1.jpg")
    print("        image2.jpg")
    print("    person2_name/")
    print("        image1.jpg")
else:
    print(f"Loaded {len(known_faces)} people:")  # len(known_faces) is the number of people in the known_faces dictionary
    for person, encodings in known_faces.items():
        print(f"- {person}: {len(encodings)} training images") # Ajai: 3 training images


No face found in c:\Users\ajair\Desktop\github\AiSystemLabs\face_recognition_project\known_faces\Ajai_Raj\image1.jpeg
Loaded face from c:\Users\ajair\Desktop\github\AiSystemLabs\face_recognition_project\known_faces\Ajai_Raj\image2.jpeg
Loaded face from c:\Users\ajair\Desktop\github\AiSystemLabs\face_recognition_project\known_faces\Ajai_Raj\image3.jpeg
No face found in c:\Users\ajair\Desktop\github\AiSystemLabs\face_recognition_project\known_faces\Ajai_Raj\image4.jpeg
Loaded face from c:\Users\ajair\Desktop\github\AiSystemLabs\face_recognition_project\known_faces\Ajai_Raj\image5.jpeg
No face found in c:\Users\ajair\Desktop\github\AiSystemLabs\face_recognition_project\known_faces\Ajai_Raj\image6.jpeg
Loaded face from c:\Users\ajair\Desktop\github\AiSystemLabs\face_recognition_project\known_faces\Ajai_Raj\image7.jpeg
Loaded 1 people:
- Ajai_Raj: 4 training images


In [5]:
# Function to process a single frame
def process_frame(frame, known_faces):
    # Find all face locations in the current frame
    face_locations = face_recognition.face_locations(frame)
    # Get face encodings for any faces in the frame
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    
    # Process each face in the frame
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        name = "Unknown"
        confidence = 0
        
        # Check against each person's face encodings
        for person_name, person_encodings in known_faces.items():
            # Compare against all training images for this person
            matches = face_recognition.compare_faces(person_encodings, face_encoding, tolerance=0.6)
            if True in matches:
                # Calculate confidence
                match_confidence = matches.count(True) / len(matches)
                if match_confidence > confidence:
                    confidence = match_confidence
                    name = person_name
        
        # Draw a box around the face
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        
        # Draw a label with the name and confidence
        label = f"{name} ({confidence*100:.0f}%)" if name != "Unknown" else name
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, label, (left + 6, bottom - 6), font, 0.6, (255, 255, 255), 1)
    
    return frame


In [6]:
# Main loop for face recognition
try:
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        if not ret:
            print("Failed to grab frame")
            break
            
        # Process the frame
        frame = process_frame(frame, known_faces)
            
        # Display the resulting frame
        cv2.imshow('Video', frame)
            
        # Hit 'q' on the keyboard to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
except KeyboardInterrupt:
    print("\nStopping the program...")
    
finally:
    # Release handle to the webcam
    video_capture.release()
    cv2.destroyAllWindows()
